In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

# Configuración
plt.style.use('seaborn-v0_8-whitegrid')
pd.set_option('display.max_columns', 50)

# Rutas
BASE_DIR = Path().resolve().parent
DATA_PATH = BASE_DIR / 'data' / 'gold' / 'model' / 'regression_monthly_dataset.parquet'

In [ ]:
# Cargar datos
df = pd.read_parquet(DATA_PATH)
print(f"Shape: {df.shape}")
print(f"\nColumnas ({len(df.columns)}):")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2}. {col}")
df.head()

In [ ]:
# Info general
print("=" * 60)
print("INFO GENERAL")
print("=" * 60)
print(f"\nTotal registros: {len(df):,}")
print(f"Período: {df['anio'].min()} - {df['anio'].max()}")
print(f"Municipios: {df['codigo_municipio'].nunique()}")
print(f"Meses cubiertos: {df.groupby(['anio', 'mes']).ngroups}")

# Valores nulos
print(f"\nValores nulos (top 10):")
nulls = df.isnull().sum().sort_values(ascending=False)
print(nulls[nulls > 0].head(10) if any(nulls > 0) else "Ninguno")

## 1. Distribución de total_delitos

In [ ]:
# Estadísticas del target principal
print("=" * 60)
print("ESTADÍSTICAS: total_delitos")
print("=" * 60)
print(df['total_delitos'].describe())

# Percentiles adicionales
print(f"\nPercentiles adicionales:")
for p in [1, 5, 95, 99]:
    print(f"  P{p}: {df['total_delitos'].quantile(p/100):.0f}")

In [ ]:
# Visualización de la distribución
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Histograma original
axes[0, 0].hist(df['total_delitos'], bins=50, color='steelblue', edgecolor='black')
axes[0, 0].set_xlabel('Total Delitos')
axes[0, 0].set_ylabel('Frecuencia')
axes[0, 0].set_title('Distribución Original')
axes[0, 0].axvline(df['total_delitos'].mean(), color='red', linestyle='--', label=f'Media: {df["total_delitos"].mean():.0f}')
axes[0, 0].axvline(df['total_delitos'].median(), color='green', linestyle='--', label=f'Mediana: {df["total_delitos"].median():.0f}')
axes[0, 0].legend()

# Log-transform
df_log = np.log1p(df['total_delitos'])
axes[0, 1].hist(df_log, bins=50, color='coral', edgecolor='black')
axes[0, 1].set_xlabel('log(Total Delitos + 1)')
axes[0, 1].set_ylabel('Frecuencia')
axes[0, 1].set_title('Distribución Log-transformada')

# Boxplot por año
df.boxplot(column='total_delitos', by='anio', ax=axes[1, 0])
axes[1, 0].set_title('Distribución por Año')
axes[1, 0].set_xlabel('Año')
axes[1, 0].tick_params(axis='x', rotation=45)
plt.suptitle('')

# Boxplot por mes
df.boxplot(column='total_delitos', by='mes', ax=axes[1, 1])
axes[1, 1].set_title('Distribución por Mes')
axes[1, 1].set_xlabel('Mes')
plt.suptitle('')

plt.tight_layout()
plt.show()

## 2. Análisis de Estacionalidad

In [ ]:
# Estacionalidad mensual
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Promedio por mes (todos los años)
monthly_avg = df.groupby('mes')['total_delitos'].mean()
axes[0].bar(monthly_avg.index, monthly_avg.values, color='steelblue')
axes[0].set_xlabel('Mes')
axes[0].set_ylabel('Promedio Delitos')
axes[0].set_title('Promedio de Delitos por Mes')
axes[0].set_xticks(range(1, 13))
axes[0].axhline(monthly_avg.mean(), color='red', linestyle='--', label='Promedio general')
axes[0].legend()

# Tendencia anual
yearly_total = df.groupby('anio')['total_delitos'].sum()
axes[1].plot(yearly_total.index, yearly_total.values, marker='o', linewidth=2, color='coral')
axes[1].set_xlabel('Año')
axes[1].set_ylabel('Total Delitos')
axes[1].set_title('Tendencia Anual de Delitos')
axes[1].grid(True)

plt.tight_layout()
plt.show()

# Coeficiente de variación por mes
print("\nCoeficiente de Variación por Mes:")
cv_by_month = df.groupby('mes')['total_delitos'].agg(['mean', 'std'])
cv_by_month['cv'] = (cv_by_month['std'] / cv_by_month['mean'] * 100).round(1)
print(cv_by_month)

In [ ]:
# Heatmap de delitos por año-mes
pivot = df.groupby(['anio', 'mes'])['total_delitos'].sum().unstack()

fig, ax = plt.subplots(figsize=(14, 8))
sns.heatmap(pivot, cmap='YlOrRd', annot=False, fmt='.0f', ax=ax)
ax.set_title('Heatmap de Delitos por Año-Mes (Total Departamento)')
ax.set_xlabel('Mes')
ax.set_ylabel('Año')
plt.tight_layout()
plt.show()

## 3. Correlaciones

In [ ]:
# Identificar features numéricas (excluyendo targets de tasas para claridad)
exclude_cols = ['codigo_municipio'] + [c for c in df.columns if 'tasa_' in c]
numeric_cols = [c for c in df.select_dtypes(include=[np.number]).columns if c not in exclude_cols]

# Top 15 features
top_features = numeric_cols[:15]

fig, ax = plt.subplots(figsize=(12, 10))
corr_matrix = df[top_features].corr()
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r', 
            center=0, ax=ax, annot_kws={'size': 8})
ax.set_title('Matriz de Correlación (Top 15 Features)')
plt.tight_layout()
plt.show()

In [ ]:
# Correlaciones más fuertes con total_delitos
print("=" * 60)
print("CORRELACIONES CON total_delitos")
print("=" * 60)

all_numeric = df.select_dtypes(include=[np.number]).columns
corr_with_target = df[all_numeric].corr()['total_delitos'].drop('total_delitos').sort_values(ascending=False)

print("\nTop 10 positivas:")
print(corr_with_target.head(10).round(3))
print("\nTop 10 negativas:")
print(corr_with_target.tail(10).round(3))

## 4. Análisis de Features Derivadas

In [ ]:
# Verificar si hay lag features
lag_cols = [c for c in df.columns if 'lag' in c.lower()]
print("Lag features encontradas:")
print(lag_cols if lag_cols else "Ninguna - Se recomienda crearlas")

# Verificar features de tendencia
trend_cols = [c for c in df.columns if any(x in c.lower() for x in ['trend', 'roll', 'pct_change'])]
print("\nFeatures de tendencia encontradas:")
print(trend_cols if trend_cols else "Ninguna - Se recomienda crearlas")

In [ ]:
# Distribución de features de tipo de delito (si existen)
delito_cols = [c for c in df.columns if c in ['ABIGEATO', 'AMENAZAS', 'DELITOS SEXUALES', 
                                               'EXTORSION', 'HOMICIDIOS', 'HURTOS', 
                                               'LESIONES', 'VIOLENCIA INTRAFAMILIAR']]

if delito_cols:
    fig, ax = plt.subplots(figsize=(12, 6))
    df[delito_cols].sum().sort_values(ascending=True).plot(kind='barh', ax=ax, color='steelblue')
    ax.set_xlabel('Total de Casos')
    ax.set_title('Distribución por Tipo de Delito')
    plt.tight_layout()
    plt.show()
else:
    print("No se encontraron columnas de tipos de delito individuales")

## 5. Conclusiones y Recomendaciones

In [ ]:
print("=" * 60)
print("CONCLUSIONES")
print("=" * 60)

skewness = df['total_delitos'].skew()
zeros = (df['total_delitos'] == 0).sum()

print(f"""
DISTRIBUCIÓN DEL TARGET:
  - Registros: {len(df):,}
  - Media: {df['total_delitos'].mean():.1f}
  - Mediana: {df['total_delitos'].median():.0f}
  - Skewness: {skewness:.2f} ({'Alta asimetría' if abs(skewness) > 1 else 'Moderada'})
  - Valores en cero: {zeros} ({zeros/len(df)*100:.1f}%)

ESTACIONALIDAD:
  - {'Se detecta patrón mensual' if monthly_avg.std() / monthly_avg.mean() > 0.1 else 'Patrón mensual débil'}
  - Mes con más delitos: {monthly_avg.idxmax()}
  - Mes con menos delitos: {monthly_avg.idxmin()}

FEATURES:
  - Total features: {len(df.columns)}
  - Lag features: {'Sí' if lag_cols else 'No (crear)'}
  - Trend features: {'Sí' if trend_cols else 'No (crear)'}
""")

print("\n" + "=" * 60)
print("RECOMENDACIONES PARA MODELO")
print("=" * 60)
print("""
1. TRANSFORMACIÓN:
   - Aplicar log1p() al target si skewness > 1
   - Considerar modelos que manejen zeros

2. FEATURE ENGINEERING:
   - Crear lag features: lag_1, lag_3, lag_12
   - Rolling means: roll_3, roll_6, roll_12
   - Cambio porcentual: pct_change_1, pct_change_12
   - Encodings temporales: mes_sin, mes_cos

3. MODELOS SUGERIDOS:
   - LightGBM / XGBoost (mejor para datos tabulares)
   - Random Forest
   - Prophet (para serie temporal departamental)

4. VALIDACIÓN:
   - TimeSeriesSplit (respetando temporalidad)
   - No usar KFold aleatorio (data leakage)

5. MÉTRICAS:
   - RMSE, MAE en escala original
   - MAPE (cuidado con zeros)
   - R² para varianza explicada
""")